In [6]:
!pip install vtk
!pip install plotly
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [19]:
import vtk

# Loading dataset given to us
reader = vtk.vtkXMLImageDataReader()
file_path = './mixture.vti'
reader.SetFileName(file_path)
reader.Update()
image_data = reader.GetOutput()

In [20]:
import numpy as np
from vtk.util import numpy_support

# Getting the scalar data from the VTK dataset
scalar_data = image_data.GetPointData().GetScalars()
data_dim = image_data.GetDimensions()

# Converting the VTK scalar data to a NumPy array
numpy_array = numpy_support.vtk_to_numpy(scalar_data)

# Reshaping the scalar data as per required encoding for iso_surface creation
np_data = numpy_array.reshape(data_dim, order='F') # order='F' because plotly uses fortron like structure


In [21]:
X, Y, Z = np.mgrid[0:data_dim[0], 0:data_dim[1], 0:data_dim[2]] # creating mesh grid for plotting given 3D data

In [22]:
# import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets

# make iso-surface object 
isosurface = go.Isosurface(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=np_data.flatten(),
    isomin=0.0,
    isomax=0.0,
    cauto = False,
    cmin = np_data.min(),
    cmax = np_data.max(),
    colorscale='plasma',
    showscale=False
)

# layout of plotly graphical figure objects
layout = go.Layout(
    scene=dict(
        xaxis=dict(showticklabels=False),
        yaxis=dict(showticklabels=False),
        zaxis=dict(showticklabels=False)
    )
)

# creating iso_surface figur widget object and setting the layout
iso_fig = go.FigureWidget(data=[isosurface], layout=layout)
iso_fig.update_layout(width=520, height=500) # set height and width of iso_surface figure

# creating histogram figure widget object and setting the layout 
hist_fig = go.Histogram(x=np_data.flatten(), nbinsx=30)
histogram = go.FigureWidget(hist_fig)
histogram.update_layout(width=500, height=500, showlegend=False, xaxis_title = 'Vortex scalar values', yaxis_title = 'Frequency')

# creating a floating point slider widget for changing the value of iso_value for iso_surface updation
slider = widgets.FloatSlider(
    value=0.0,
    min=np_data.min(),
    max=np_data.max(),
    step=0.01,
    description='Isovalue:',
    disabled=False,
    continuous_update=False,
)

# Create Reset button for resetting to defaults 
reset_button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style=''
)

# Records button action when interated by user
def on_button_click(reset_button):
    slider.value = 0.00
    isovalue = 0.00
    iso_fig.data[0].isomin= isovalue
    iso_fig.data[0].isomax= isovalue
    histogram.data[0].x = np_data.flatten() 

# update isosurface function to be passed to slider interaction
def update_isosurface(isovalue):
    isomin = isovalue.new
    isomax = isovalue.new
    iso_fig.data[0].isomin = isomin
    iso_fig.data[0].isomax = isomax
    filtered_values = [val for val in np_data.flatten() if val >= isovalue.new - 0.25 and val <= isovalue.new + 0.25]
    histogram.data[0].x = filtered_values

# observes interaction to slider by user and do the action 
slider.observe(update_isosurface, names='value')

# captures reset button interaction
reset_button.on_click(on_button_click)


# For creating the layout as given in assignment pdf -> VBox and HBox containers of widgets have been used
Overall_vis = widgets.VBox([
    widgets.HBox([slider, reset_button]), 
    widgets.HBox([iso_fig,histogram])
])

# display the final visualization 
display(Overall_vis)